In [57]:
# https://adrien.barbaresi.eu/blog/trafilatura-main-text-content-python.html

import requests
import bs4
from bs4 import BeautifulSoup
import os


# Get archives

In [79]:
page_url = "https://kghosh.substack.com/archive"

page_sourced = requests.get(page_url).content 

html_content = BeautifulSoup(page_sourced, "html.parser")
# #main > div.archive-page.typography.use-theme-bg > div > div > div.portable-archive-list > div:nth-child(19) > div.post-preview-content > a.post-preview-title.newsletter
links = html_content.findAll('a', class_="post-preview-title newsletter")
print(len(links),links) 
pages = [i.get('href') for i in links if (not (i.get('href') is None))]

pages = [ i for i in pages  if "https://kghosh.substack.com/p/" in i ]
pages = list(set([ i for i in pages  if not i.endswith("comments") ]))
len(pages), pages

11 [<a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221002">20221002</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220922">20220922</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220917">20220917</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/borges-babel-movies-a-short-story">Borges Babel Movies - a short story</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220904">20220904</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220828">20220828</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220824">20220824</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220721">20220721</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220717">20220717</a>, <a class="post-preview-title newsle

(11,
 ['https://kghosh.substack.com/p/20221002',
  'https://kghosh.substack.com/p/20220922',
  'https://kghosh.substack.com/p/20220917',
  'https://kghosh.substack.com/p/borges-babel-movies-a-short-story',
  'https://kghosh.substack.com/p/20220904',
  'https://kghosh.substack.com/p/20220828',
  'https://kghosh.substack.com/p/20220824',
  'https://kghosh.substack.com/p/20220721',
  'https://kghosh.substack.com/p/20220717',
  'https://kghosh.substack.com/p/20220711',
  'https://kghosh.substack.com/p/20220626'])

In [88]:
for page in pages:
    name = page.split("/")[-1]

    if not os.path.exists(".cache/"+name):
        page_sourced = requests.get(page).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        content = html_content.findAll('div', class_="body markup")

        with open(".cache/"+name, 'w') as f:
            f.write(str(content))
        print(name,"saved")
    else:
        print(name,"exists.")

20220626 exists.
20220828 exists.
20220711 exists.
20220917 exists.
20221002 exists.
borges-babel-movies-a-short-story exists.
20220721 exists.
20220904 exists.
20220922 exists.
20220810 exists.
20220717 exists.
20220824 exists.


In [89]:
import glob
import pandas as pd


cached_pages = glob.glob(".cache/*")


In [99]:
URLs = []
for page in cached_pages:
    with open(page) as fp:
        html_content = BeautifulSoup(fp, 'html.parser')  
    content = html_content.findAll('a')
    content = [i.get('href') for i in content if (not (i.get('href') is None))]
    for link in content:
        URLs.append([name,link])
    print(content) 

['https://spectrum.ieee.org/tesla-autopilot-data-ownership', 'https://themarkup.org/the-breakdown/2022/07/27/who-is-collecting-data-from-your-car?utm_source=newsletter&utm_medium=email&utm_campaign=newsletter_axioswhatsnext&stream=science', 'https://www.cnil.fr/en/qa-cnils-formal-notices-concerning-use-google-analytics', 'https://techcrunch.com/2022/07/18/instagrams-buy-products-directly-dms/', 'https://twitter.com/Wendigoon8/status/1549839412531531780', 'https://www.foreground.com.au/environment/designing-without-depletion-joseph-grima/', 'https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F9a5e2655-4740-41df-8066-4b9788624724_2268x1792.jpeg?utm_source=substack&utm_medium=email', 'https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2Fbdbbe840-bd49

In [113]:
import hashlib

df = pd.DataFrame(URLs)
df.columns = ["page","url"]
df = df.drop_duplicates(subset=["url"])
df["url"] = df.url.apply(lambda x: str(x).encode('utf-8'))

df["hash"] = df.url.apply(lambda x: hashlib.md5(str(x).encode('utf-8')).hexdigest())
df.to_parquet("urls.parquet.gzip",  engine='pyarrow', compression='gzip')
df

,page,url,hash
0,20220824,b'https://spectrum.ieee.org/tesla-autopilot-da...,aa8144a7a9cdc5baa399df0daa112f75
1,20220824,b'https://themarkup.org/the-breakdown/2022/07/...,fe7e41280ce4475f799785a436070868
2,20220824,b'https://www.cnil.fr/en/qa-cnils-formal-notic...,903096f9f71209541fb6fabd6220b72b
3,20220824,b'https://techcrunch.com/2022/07/18/instagrams...,fd5ff8626bba470fb179c76d3153d165
4,20220824,b'https://twitter.com/Wendigoon8/status/154983...,a95cf828f35be5e3739324cfde416207
...,...,...,...
283,20220824,b'https://www.goodreads.com/fr/book/show/25478...,38438ce40cddf2d58499fd70b4ffc7a6
284,20220824,b'https://en.wikipedia.org/wiki/Three_Thousand...,51b141d1f32190e4f3c60c8fa01ddb46
285,20220824,b'https://www.goodreads.com/book/show/45154547...,48cdce80ccaf0dc29c895755ae7f51c0
286,20220824,b'https://kenliu.name/blog/book/the-paper-mena...,6349bb23de426f5481fb2427dbc79efb
